<h1>XGBoost</h1>

In [1]:
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score

# scoring functions for feature selection
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile

from sklearn.metrics import make_scorer

import csv
import os

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
class XGBoost():
    """A class to choose and train an XGBoost model"""
    """This implements the XGBoost model"""
    
    def __init__(self, data_dir='./processed_data/', regressor=False, n_jobs=1,
                 criterion=None, random_state=1, n_estimators=1000,
                 opt_func=None, inv_opt_func=None, scorer=None):
        """Initializes the XGBoost class"""
        
        # TODO:
        #    - Create an initialization function that will allow for
        #      a saved model to be loaded
        
        self.n_jobs = n_jobs
        self.regressor = regressor
        self.data_dir = data_dir
        self.regressor = regressor
        self.criterion = criterion
        self.random_state = random_state
        self.n_estimators = n_estimators
        self.opt_func = opt_func
        self.inv_opt_func = inv_opt_func
        self.scorer = scorer
        
        # keep track of which functions have been called
        self.read_data_called = False
        self.select_features_called = False
        self.tune_params_called = False
        self.train_model_called = False
        
        if criterion == None:
            if regressor:
                self.criterion = 'reg:linear'
            else:
                self.criterion = 'binary:logistic'
        
    def read_data(self):
        """Read in the data from the specified directory"""
        
        self.read_data_called = True
        
        self.cv_X_df = pd.read_csv(self.data_dir+'cv_X.csv', header=0)
        self.cv_y_df = pd.read_csv(self.data_dir+'cv_y.csv', header=0)
        self.cv_ids_df = pd.read_csv(self.data_dir+'cv_ids.csv', header=0)
        self.train_X_df = pd.read_csv(self.data_dir+'train_X.csv', header=0)
        self.train_y_df = pd.read_csv(self.data_dir+'train_y.csv', header=0)
        self.train_ids_df = pd.read_csv(self.data_dir+'train_ids.csv', header=0)
        self.test_X_df = pd.read_csv(self.data_dir+'test_X.csv', header=0)
        self.test_ids_df = pd.read_csv(self.data_dir+'test_ids.csv', header=0)
        
    def get_feature_importances(self):
        """Trains a basic random forest to get a list of feature importances"""
        
        if not(self.read_data_called):
            raise AssertionError("No data yet!")
        
        importance_tree = RandomForestRegressor(n_jobs=self.n_jobs,
                                random_state=self.random_state,
                                n_estimators=self.n_estimators)
        
        # train a basic model so that we can access feature importances
        cv_X = self.cv_X_df.values
        cv_y = np.ravel(self.cv_y_df.values)
        importance_tree.fit(cv_X, cv_y)
        self.feature_importances = importance_tree.feature_importances_
        
        if not(self.opt_func == None):
            cv_y = self.opt_func(cv_y)
        
        # sort the features by importances, most important first
        self.sorted_features = self.feature_importances.argsort()[::-1]
        
        return self.sorted_features
        
    '''    
    def select_features(self):
        """Select features from the dataset by using feature importances
        to add features one by one for selection"""
        
        if not(self.read_data_called):
            raise AssertionError("No data yet!")
            
        self.select_features_called = True
            
        # TODO:
        #   - VERIFY THAT THIS IS WORKING
        #   - THINK OF OTHER TYPES OF FEATURE SELECTION
        #     BACKWARD FEATURE SELECTION?
        #   - IS THIS THE ONLY CONSIDERATION FOR SELECTING FEATURES??
        #     WHAT IF THE SCORE IS VERY HIGH WITH FEW FEATURES OUT OF MANY??
        #     WHAT IF THE LARGEST SCORE IS AFTER MANY FEATURES ARE ADDED
        #     WITH MINIMAL IMPROVEMENT TO SCORE??
        
        if self.regressor:
            model = RandomForestRegressor(oob_score=True,
                                        n_jobs=self.n_jobs,
                                        random_state=self.random_state,
                                        n_estimators=self.n_estimators,
                                        criterion=self.criterion)
        else:
            model = RandomForestClassifier(oob_score=True,
                                         n_jobs=self.n_jobs,
                                         random_state=self.random_state,
                                         n_estimators=self.n_estimators)
            
        # keep track of the scores from cross validation
        all_scores = []
        max_score = -10000
        early_stopping_rounds = 10
        early_stopping_count = 0
        
        # which features to add, in order
        self.sorted_features = self.get_feature_importances()
        
        cv_X = self.cv_X_df.values
        cv_y = np.ravel(self.cv_y_df.values)
        cv_X_selected = np.array([])
        cv_X_selected.shape = (cv_X.shape[0], 0)
                
        if not(self.opt_func == None):
            cv_y = self.opt_func(cv_y)
            
        for feature in self.sorted_features:
            
            # format data so that it can be added easily
            added_feat = np.transpose(cv_X[:, feature])
            added_feat.shape += (1, )
            
            # append data to the next column
            cv_X_selected = np.append(cv_X_selected, added_feat, axis=1)
            
            # train a model with the augmented data
            #model.fit(cv_X_selected, cv_y)
            #score = model.oob_score_
            scores = cross_val_score(model, cv_X_selected, cv_y, cv=3, scoring='neg_mean_squared_error')
            score = scores.mean()
            print(feature)
            print(score)
            
            # have an evaluation metric to measure performance of added features
            all_scores.append(score)
            
            if score > max_score:
                max_score = score
                early_stopping_count = 0
            else:
                early_stopping_count += 1
                
            if early_stopping_count > early_stopping_rounds:
                break
        
        max_score_index = np.argmax(all_scores)
        print(all_scores)
        print(max_score_index)
        
        # select the features
        self.selected_feature_indices = self.sorted_features[:(max_score_index+1)]
        self.selected_features = cv_X_selected[:, :(max_score_index+1)]
        
        return self.selected_feature_indices
    '''
    
    
    def select_features_2(self, score_func_name='f_regression', percentage=100):
        """Function to select features based on sklearn scoring"""
        
        self.select_features_called = True
        
        cv_X = self.cv_X_df.values
        cv_y = np.ravel(self.cv_y_df.values)
        
        if score_func_name == 'chi2':
            score_func = chi2
        elif score_func_name == 'f_regression':
            score_func = f_regression
        elif score_func_name == 'f_classif':
            score_func = f_classif
        
        if not(self.opt_func == None):
            cv_y = self.opt_func(cv_y)
        
        self.feature_selector = SelectPercentile(score_func, percentile=percentage)
        self.selected_features = self.feature_selector.fit_transform(cv_X, cv_y)
        
        return self.selected_features
        
    def tune_params(self):
        """Function to handle tuning hyperparameters of the model"""
        
        # TODO:
        #   - FIND GOOD VALUES OF PARAMETERS TO TEST
        
        if not(self.read_data_called):
            raise AssertionError("No data yet!")
        
        self.tune_params_called = True
        
        # tree structure
        self.tune_max_depth = True
        self.tune_min_child_weight = True
        
        self.tune_gamma = True
        
        self.tune_subsample = True
        self.tune_colsample_bytree = True
        
        self.tune_reg_alpha = True
        self.tune_reg_lambda = True
        
        # Parameters to tune for XGBoost
        # the maximum number of features to be considered for a tree.
        #max_depth_pos = [2, 4, 6, 8, 10, 12, 20]
        #min_child_weight_pos = [1, 2, 4, 6, 8, 10]
        max_depth_pos = [6, 10, 20]
        min_child_weight_pos = [1, 3, 5]
        
        #gamma_pos = [0, 0.001, 0.01, 0.1, 1]
        gamma_pos = [0, 0.01, 1]
        
        #subsample_pos = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
        #colsample_bytree_pos = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
        subsample_pos = [0.5, 0.75, 1]
        colsample_bytree_pos = [0.5, 0.75, 1]
        
        #reg_alpha_pos = [0, 0.001, 0.01, 0.1, 1]
        #reg_lambda_pos = [0, 0.001, 0.01, 0.1, 1]        
        reg_alpha_pos = [0, 0.01, 1]
        reg_lambda_pos = [0, 0.01, 1]
        
        # TODO:
        #    - FIGURE OUT WHICH OTHER PARAMETERS TO TUNE
        #    - DETERMINE GOOD VALUES TO TEST
        
        # set these parameters to commonly used starting values
        self.max_depth=5 # default=3
        learning_rate_tune=0.1
        self.silent=False
        self.gamma=0
        self.min_child_weight=1
        self.max_delta_step=0
        self.subsample=0.8 #default=1
        self.colsample_bytree=0.8 # default=1
        self.colsample_bylevel=1
        self.reg_alpha=0
        self.reg_lambda=1
        self.scale_pos_weight=1
        
        # training parameters
        early_stopping_rounds = 20
        n_estimators_tune = self.n_estimators
        cv_splits = 5

        
        # only take the selected features if we have performed training already
        if self.select_features_called:
            cv_X = self.selected_features
        else:
            cv_X = self.cv_X_df.values
        
        cv_y = np.ravel(self.cv_y_df.values)
        
        if not(self.opt_func == None):
            cv_y = self.opt_func(cv_y)
        
        if self.regressor:
            model = xgb.XGBRegressor(max_depth=self.max_depth,
                                learning_rate=learning_rate_tune,
                                n_estimators=n_estimators_tune,
                                silent=self.silent,
                                objective=self.criterion,
                                nthread=self.n_jobs,
                                gamma=self.gamma,
                                min_child_weight=self.min_child_weight,
                                max_delta_step=self.max_delta_step,
                                subsample=self.subsample,
                                colsample_bytree=self.colsample_bytree,
                                colsample_bylevel=self.colsample_bylevel,
                                reg_alpha=self.reg_alpha,
                                reg_lambda=self.reg_lambda,
                                scale_pos_weight=self.scale_pos_weight,
                                seed=self.random_state)
        else:
            model = xgb.XGBClassifier(max_depth=self.max_depth,
                                learning_rate=learning_rate_tune,
                                n_estimators=n_estimators_tune,
                                silent=self.silent,
                                objective=self.criterion,
                                nthread=self.n_jobs,
                                gamma=self.gamma,
                                min_child_weight=self.min_child_weight,
                                max_delta_step=self.max_delta_step,
                                subsample=self.subsample,
                                colsample_bytree=self.colsample_bytree,
                                colsample_bylevel=self.colsample_bylevel,
                                reg_alpha=self.reg_alpha,
                                reg_lambda=self.reg_lambda,
                                scale_pos_weight=self.scale_pos_weight,
                                seed=self.random_state)
                    
        # perform training first to get a baseline and number
        # of training rounds
        # GET EARLY STOPPING TO WORK FOR THIS
        #model.fit(cv_X, cv_y)
        #n_estimators_tune = model.best_iteration
        #best_score = model.best_score
        model.set_params(n_estimators=n_estimators_tune)
        
        
        # perform parameter-spac e searching
        if self.tune_max_depth and self.tune_min_child_weight:
            scores = []
            params = []
            for max_depth in max_depth_pos:
                model.set_params(max_depth=max_depth)
                for min_child_weight in min_child_weight_pos:
                    model.set_params(min_child_weight=min_child_weight)
                    print(model.get_params)
                    score = cross_val_score(model, cv_X, cv_y, cv=cv_splits,
                                         scoring=make_scorer(self.scorer))
                    score = score.mean()
                    scores.append(score)
                    params.append((max_depth, min_child_weight))
                    print(score)
            
            # select these indices
            best_param_index = np.argmax(scores)
            print(best_param_index)
            self.max_depth = params[best_param_index][0]
            self.min_child_weight = params[best_param_index][1]
            model.set_params(max_depth=self.max_depth)
            model.set_params(min_child_weight=self.min_child_weight)
            
        if self.tune_gamma:
            scores = []
            params = []
            for gamma in gamma_pos:
                model.set_params(gamma=gamma)
                print(model.get_params)
                score = cross_val_score(model, cv_X, cv_y, cv=cv_splits,
                                     scoring=make_scorer(self.scorer))
                score = score.mean()
                scores.append(score)
                params.append(gamma)
                print(score)
            
            best_param_index = np.argmax(scores)
            print(best_param_index)
            self.gamma = params[best_param_index]
            model.set_params(gamma=self.gamma)
                    
        if self.tune_subsample and self.tune_colsample_bytree:
            scores = []
            params = []
            for subsample in subsample_pos:
                model.set_params(subsample=subsample)
                for colsample_bytree in colsample_bytree_pos:
                    model.set_params(colsample_bytree=colsample_bytree)
                    print(model.get_params)
                    score = cross_val_score(model, cv_X, cv_y, cv=cv_splits,
                                         scoring=make_scorer(self.scorer))
                    score = score.mean()
                    scores.append(score)
                    params.append((subsample, colsample_bytree))
                    print(score)
            
            best_param_index = np.argmax(scores)
            print(best_param_index)
            self.subsample = params[best_param_index][0]
            self.colsample_bytree = params[best_param_index][1]
            model.set_params(subsample=self.subsample)
            model.set_params(colsample_bytree=self.colsample_bytree)

        if self.tune_reg_alpha and self.tune_reg_lambda:
            scores = []
            params = []
            for reg_alpha in reg_alpha_pos:
                model.set_params(reg_alpha=reg_alpha)
                for reg_lambda in reg_lambda_pos:
                    model.set_params(reg_lambda=reg_lambda)
                    print(model.get_params)
                    score = cross_val_score(model, cv_X, cv_y, cv=cv_splits,
                                         scoring=make_scorer(self.scorer))
                    score = score.mean()
                    scores.append(score)
                    params.append((reg_alpha, reg_lambda))
                    print(score)
            
            best_param_index = np.argmax(scores)
            print(best_param_index)
            self.reg_alpha = params[best_param_index][0]
            self.reg_labmda = params[best_param_index][1]
            model.set_params(reg_alpha=self.reg_alpha)
            model.set_params(reg_lambda=self.reg_lambda)
        
        self.model = model
        
    def train_model(self, n_estimators=10000, learning_rate=0.01):
        """Trains the model on all training data
        and returns a model to be used for prediction"""
        
        if not(self.read_data_called):
            raise AssertionError("No data yet!")
            
        self.train_model_called = True
        
        
            
        # determine which model to use
        if self.tune_params_called:
            self.model = self.model
            self.model.set_params(n_estimators=n_estimators)
        else:
            if self.regressor:
                model = xgb.XGBRegressor(max_depth=self.max_depth,
                                learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                silent=self.silent,
                                objective=self.criterion,
                                nthread=self.n_jobs,
                                gamma=self.gamma,
                                min_child_weight=self.min_child_weight,
                                max_delta_step=self.max_delta_step,
                                subsample=self.subsample,
                                colsample_bytree=self.colsample_bytree,
                                colsample_bylevel=self.colsample_bylevel,
                                reg_alpha=self.reg_alpha,
                                reg_lambda=self.reg_lambda,
                                scale_pos_weight=self.scale_pos_weight,
                                seed=self.random_state)
            else:
                model = xgb.XGBClassifier(max_depth=self.max_depth,
                                learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                silent=self.silent,
                                objective=self.criterion,
                                nthread=self.n_jobs,
                                gamma=self.gamma,
                                min_child_weight=self.min_child_weight,
                                max_delta_step=self.max_delta_step,
                                subsample=self.subsample,
                                colsample_bytree=self.colsample_bytree,
                                colsample_bylevel=self.colsample_bylevel,
                                reg_alpha=self.reg_alpha,
                                reg_lambda=self.reg_lambda,
                                scale_pos_weight=self.scale_pos_weight,
                                seed=self.random_state)
        
        if self.select_features_called:
            train_X = self.selected_features
        else:
            train_X = self.train_X_df.values
            
        train_y = np.ravel(self.train_y_df.values)
                        
        if not(self.opt_func == None):
            train_y = self.opt_func(train_y)
            
        print(self.model.get_params)
        
        self.model.fit(train_X, train_y)
        
    def predict_output(self):
        """Make predictions on test data."""
        
        if not(self.train_model_called):
            raise AssertionError("Train the model first!")
        
        # transform the features to match the selected features from training
        #test_X_all = self.test_X_df.values
        #test_X = np.array([])
        #test_X.shape = (test_X_all.shape[0], 0)
        #for feature in self.selected_feature_indices:
            #
            # format data so that it can be added easily
            #added_feat = np.transpose(test_X_all[:, feature])
            #added_feat.shape += (1, )
            #
            # append data to the next column
            #test_X = np.append(test_X, added_feat, axis=1)
            
        
        test_X = self.test_X_df.values
        test_X = self.feature_selector.transform(test_X)
        
        self.pred = self.model.predict(test_X)
                        
        if not(self.inv_opt_func == None):
            self.pred = self.inv_opt_func(self.pred)
            
        return self.pred
    
    def write_output(self, output_file='output.csv', header=['Id', 'Result'],
                    out_dir='./output/'):
        # create output dir if it does not exist
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
            
        # write output to a file
        prediction_file = open(out_dir+output_file, 'w')
        open_file_object = csv.writer(prediction_file, lineterminator='\n')
        open_file_object.writerow(header)
        data = zip(np.ravel(self.test_ids_df.values), self.pred)
        open_file_object.writerows(data)
        prediction_file.close()
        
    def get_model(self):
        """Return the model used for prediction"""
        return self.model
        

In [13]:
def neg_rmse(y, y_pred):
    return -1*np.sqrt(np.mean((y_pred-y)**2))

def accuracy(y, y_pred):
    correct = np.sum([1 if y[x] == y_pred[x] else 0 for x in range(len(y))])
    return float(correct) / len(y)

def log_e(y):
    return np.log(y)

def log_10(y):
    return np.log10(y)

def exp_e(y):
    return np.exp(y)

def exp_10(y):
    return np.power(10, y)

In [14]:
model = XGBoost(n_jobs=3, regressor=False, criterion='binary:logistic', 
                    opt_func=None, inv_opt_func=None, scorer=accuracy,
                    n_estimators=20)
#criterion: binary:logistic or reg:linear

model.read_data()

features = model.select_features_2(score_func_name='f_regression', percentage=100)

model.tune_params()

model.train_model(n_estimators=100, learning_rate=0.1)

pred = model.predict_output()
print(pred)

model.write_output(output_file='mnist_output_XGBoost.csv', header=['DigitId', 'Label'])

train_model = model.get_model()

/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=20, nthread=3,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1, silent=False, subsample=0.8)>
0.930666864608
<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=20, nthread=3,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1, silent=False, subsample=0.8)>
0.929785818524
<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=5, missing=None, n_estim

0.965048131778
<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0.01, learning_rate=0.1, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=20, nthread=3,
       objective='binary:logistic', reg_alpha=0, reg_lambda=0.01,
       scale_pos_weight=1, seed=1, silent=False, subsample=1)>
0.964833729847
<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0.01, learning_rate=0.1, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=20, nthread=3,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1, silent=False, subsample=1)>
0.962309727139
<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0.01, learning_rate=0.1, max_delta_step=0, max_depth=20,
       min_child_weigh